In [237]:
#Tutorial 1 - Goal Keeper problem

import pandas as pd 
import subprocess
import sasoptpy as so
import os 
import time 

In [238]:
t0 = time.time()

In [239]:
df = pd.read_csv('../data/input.csv')
gk_data = df[df['element_type'] == 1].copy().reset_index() #create a set of data with only the GK position. It will the reset the index using reset()
gk_data.head() 

,index,assists,birth_date,bonus,bps,can_select,can_transact,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,4,0,NaN,0,0,False,True,0.0,0.0,0,...,38,0,0,0,4643,0,0.0,0.0,Hein,0
1,11,0,1995-09-15,10,555,False,True,100.0,100.0,13,...,82,142,4184559,0,3511572,0,0.5,25.4,Raya,3
2,21,0,1989-07-19,0,30,False,True,100.0,100.0,0,...,47,7,92955,0,261168,0,0.0,1.7,Neto,0
3,27,0,2006-03-13,0,0,False,True,NaN,NaN,0,...,71,0,4526,0,4943,0,0.0,0.0,Setford,0
4,33,0,2008-07-15,0,0,False,True,NaN,NaN,0,...,46,0,1928,0,1018,0,0.0,0.0,Porter,0


In [240]:
lowest_two_sum = gk_data['now_cost'].nlargest(2).sum()
print(lowest_two_sum)


111


In [241]:
def solve_goalkeeper_problem(gk_data, budget):
    model = so.Model(name='gk_model')
    players = gk_data.index.tolist()

    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    bench = model.add_variables(players, name='bench', vartype=so.binary)

    total_xp = so.expr_sum(lineup[p] * gk_data.loc[p, 'ep_next'] for p in players) \
               + 0.1 * so.expr_sum(bench[p] * gk_data.loc[p, 'ep_next'] for p in players)
    
    model.set_objective(-total_xp, name='total_xp_obj', sense='N')

    model.add_constraints((lineup[p] + bench[p] <= 1 for p in players), name='lineup_or_bench')
    model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup')
    model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench')
    model.add_constraint(
        so.expr_sum((lineup[p] + bench[p]) * (gk_data.loc[p, 'now_cost'] / 10) for p in players) <= budget,
        name='budget_con'
    )

    # Export model to MPS file
    model.export_mps(filename='gk.mps')

    # Run CBC solver silently
    open('solution.txt', 'w').close()
    command = ['cbc', 'gk.mps', '-solve', '-solu', 'solution.txt']

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Optional: check if CBC succeeded
    if result.returncode != 0:
        print("CBC solver failed!")
        print("stderr:", result.stderr)
        return

    # Read solution file and set variable values
    with open('solution.txt', 'r') as f:
        # reset variables to zero
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))

    # Print lineup and bench players selected
    print("\nLINEUP")
    for p in players:
        if lineup[p].get_value() > 0.5:
            print(p, gk_data.loc[p, 'web_name'])

    print("\nBENCH")
    for p in players:
        if bench[p].get_value() > 0.5:
            print(p, gk_data.loc[p, 'web_name'])


In [242]:
solve_goalkeeper_problem(gk_data, 13)

NOTE: Initialized model gk_model.

LINEUP
7 Martinez

BENCH
16 Flekken


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sasoptpy/interface/format/mps_format.py:188: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f] = df[f].replace('', np.nan)


In [243]:
t1 = time.time()
print(t0 - t1, 'seconds')

-0.31536006927490234 seconds
